**FROM DEB:**
- Testing many hypotheses on the same data set. (Maybe this could be a genetics example).

- Collecting many data sets and carrying out one test on each (A/B testing with ads)

- Entirely different people/institutions investigating a similar phenomenon - this is the example of Paul the Octopus.

- Making many decisions in the data collection, EDA process, and analysis process. This is "tip of the iceberg" phenomen.

**START SECTION**

⬇️⬇️⬇️⬇️

In [277]:
### HIDDEN
# does this make it hidden?
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stat
import pandas as pd
np.random.seed(0)

# P-hacking
As we discussed, a p-value or probability value is the chance, based on the model in the null hypothesis, that the test statistic is equal to the value that was observed in the data or is even further in the direction of the alternative. If a p-value is small, that means the tail beyond the observed statistic is small and so the observed statistic is far away from what the null predicts. This implies that the data support the alternative hypothesis better than they support the null. By convention, when we see that the p-value is below 0.05, the result is called statistically significant, and we reject the null hypothesis.

There are dangers that present itself when the p-value is misused. *P-hacking* is the act of misusing data analysis to show that patterns in data are statistically significant, when in reality they are not. This is often done by performing multiple tests on data and only focusing on the tests that return results that are significant. 

In this section, we will go over a few examples of the dangers of p-values and p-hacking.

## Multiple Hypothesis Testing


One of the biggest dangers of blindly relying on the p-value to determine "statistical significance" comes when we are just trying to find the "sexiest" results that give us "good" p-values. This is commonly done when doing "food frequency questionairres," or FFQs, to study eating habits' correlation with other characteristics (diseases, weight, etc). 
FiveThirtyEight, an online blog that focuses on opinion poll analysis among other things, made their own FFQ, and we can use their data to run our own analysis to find some silly results that can be considered "statistically significant."

In [278]:
data = pd.read_csv('raw_anonymized_data.csv')

# These are the columns that give us characteristics of FFQ-takers
characteristics = data.columns[1:27]

# These are the columns that give us the quantities/frequencies of different food the FFQ-takers ate
ffq = data.columns[27:]

In [279]:
data[characteristics].head()

,cancer,diabetes,heart_disease,belly,ever_smoked,currently_smoke,smoke_often,smoke_rarely,never_smoked,quit_smoking,...,neutralCable,noCrash,yesCrash,uhCrash,rash,cat,dog,Dems,atheist,Jewish
0,Yes,No,No,Innie,Yes,Yes,Yes,No,No,No,...,No,Yes,No,No,Yes,No,No,Yes,Yes,No
1,No,Yes,Yes,Outie,Yes,Yes,No,Yes,No,No,...,No,No,No,Yes,Yes,No,No,Yes,Yes,No
2,Yes,Yes,Yes,Innie,No,No,No,No,Yes,No,...,Yes,No,No,Yes,No,No,Yes,No,No,No
3,No,No,No,Innie,No,No,No,No,Yes,No,...,No,No,No,Yes,No,No,No,Yes,Yes,No
4,Yes,No,No,Innie,No,No,No,No,Yes,No,...,Yes,No,No,Yes,No,No,No,Yes,Yes,No


In [280]:
data[ffq].head()

,BREAKFASTSANDWICHFREQ,BREAKFASTSANDWICHQUAN,EGGSFREQ,EGGSQUAN,YOGURTFREQ,YOGURTQUAN,COTTAGECHEESEFREQ,COTTAGECHEESEQUAN,CREAMCHEESEFREQ,CREAMCHEESEQUAN,...,DT_FIBER_INSOL,DT_FIBER_SOL,DT_PROT_ANIMAL,DT_PROT_VEGETABLE,DT_NITROGEN,PHYTIC_ACID,OXALIC_ACID,COUMESTROL,BIOCHANIN_A,FORMONONETIN
0,5,1,7,3,1,3,1,2,7,3,...,7.38,1.25,75.46,16.00,14.89,365.70,318.11,0.0117,0.0658,0.00324
1,1,1,4,4,2,3,1,2,2,2,...,9.11,3.37,59.41,18.25,12.51,434.98,112.66,0.0107,0.1390,0.00743
2,3,2,4,2,5,2,2,2,6,3,...,11.56,4.74,61.49,28.46,14.45,606.43,213.41,0.0965,0.0519,0.00946
3,2,1,7,2,8,3,1,2,1,2,...,26.34,10.85,28.71,44.59,12.15,1570.07,334.08,0.2830,0.0890,0.01260
4,3,1,5,2,8,2,2,2,4,2,...,16.48,4.80,32.41,28.23,9.80,616.99,422.55,0.1630,0.0994,0.02070


In [299]:
# Do some EDA on the data so that categorical values get changed to 1s and 0s
data.replace('Yes', 1, inplace=True)
data.replace('Innie', 1, inplace=True)
data.replace('No', 0, inplace=True)
data.replace('Outie', 0, inplace=True)

# Calculate the p value between every characteristic and food frequency/quantity
pvalues = {}
for c in characteristics:
    for f in ffq:
        pvalues[stat.ttest_ind(data[c].tolist(), data[f].tolist(), equal_var=False)[1]] = (c, f)

In [300]:
sorted_values = []
for key in sorted(pvalues.keys()):
    sorted_values += [(key, pvalues[key])]

In [301]:
sorted_values[2000:2100]

[(2.3648241474577024e-26, ('Jewish', 'FRIESFREQ')),
 (2.3793938164610517e-26, ('readingMath', 'EGGSQUAN')),
 (2.4257035809442306e-26, ('smoke_rarely', 'ASH')),
 (2.5172312265496039e-26, ('right_hand', 'GROUP_SUGARYBEVG_TOTAL_FREQUENCY')),
 (2.544424004492252e-26, ('favCable', 'FRIESFREQ')),
 (2.5763300883291532e-26, ('currently_smoke', 'ASH')),
 (2.6205744857515174e-26, ('belly', 'VEGGIESFREQ')),
 (2.6365909377739635e-26, ('cancer', 'COOKEDGREENSQUAN')),
 (2.6652602262111569e-26, ('mathReading', 'MAYOQUAN')),
 (2.7289075788443869e-26, ('uhCrash', 'BROCCOLIFREQ')),
 (2.7431115431268559e-26, ('yesCrash', 'ASH')),
 (2.8108805895937193e-26, ('cancer', 'BERRIESQUAN')),
 (2.845061863498948e-26, ('belly', 'V_REDOR_OTHER')),
 (2.9205917867356232e-26, ('belly', 'FATRNP')),
 (3.0137708614710861e-26, ('Jewish', 'SLOWWALKTIME')),
 (3.0137708614710866e-26, ('smoke_often', 'SLOWWALKTIME')),
 (3.0275989228139054e-26, ('rash', 'ASH')),
 (3.1014395638929791e-26, ('right_hand', 'PSVEGPOT')),
 (3.1326215

In [ ]:
As you can see, a lot of 

## A/B Testing
A/B testing is a very simple concept. We measure a statistic in a normal, controlled environment (we'll call this A), and then we compare that to the same statistic in an environment with *one* change. This form of testing is used frequently in marketing and ad research to compare the effectiveness of certain features of ads. 

Let's say we are working for a company whose website lets users make their own custom videogames. The company has a free version, which lets users make very basic videogames, and a paid version, which gives users access to more advanced tools for making videogames. When a user has finished making a videogame via a free account, we send them to a landing page that gives them the option to sign up for a paid account. Our measured statistic in this case would be how many free users sign up for a paid account upon reaching this page. We can send half of our users one version of the page, which may have text explaining in detail the benefits of the paid account (this will be version A), and the other half of our users will get another version of the page, which may have a colorful graphic that explains some of the benefits of the paid account (this will be version B). 

There is a very specific reason why it's called A/B testing, and not A/B/C/D... testing. That is because we can very easily run into problems if we try to test multiple versions at the same time. 

Let's say that we have 15 different sign up pages (one is the control, in this case "A"), each with something different about them (one has a picture of a puppy, one has a quote from a customer, one has a graphic, etc.), and let's say that in this case, none of our variations actually has an effect on user interaction (so we can use a Gaussian distribution with a mean of 0 and a std of 0.1).

In [275]:
n = 50
reps = 1000
num_pages = 15

landing_pages = [np.random.normal(0, 0.1, n) for _ in range(num_pages)]

# This will be our "control"
A = landing_pages[0]

# Our test statistic will be the difference between the mean percentage 
def test_stat(A, B):
    return np.abs(np.mean(B) - np.mean(A))

def permute(A, B):
    combined = np.append(A, B)
    shuffled = np.random.choice(combined, size=len(combined), replace=False)
    return shuffled[:n], shuffled[n:]

p_vals = []
for i in range(1, num_pages):
    B = landing_pages[i]
    obs = test_stat(A, B)
    resampled = [test_stat(*permute(A, B)) for _ in range(reps)]
    p_val = np.count_nonzero(obs >= resampled) / reps
    p_vals.append(p_val)
p_vals

[0.434,
 0.208,
 0.01,
 0.02,
 0.631,
 0.171,
 0.222,
 0.212,
 0.276,
 0.772,
 0.006,
 0.001,
 0.572,
 0.593]

As we can see, more than one of these ads seems to have p-values less than 0.05, despite our knowing that there actually no difference between the pages. This is why we do single A/B testing with multiple trials, as opposed to multiple hypothesis testing with only single trials. It is too easy for a p-value to give us a false positive if we just try a bunch of times.

## Many Tests for One Phenomenon
Sometimes, multiple testing can happen by accident. If many researchers are investigating the same phenomenon at the same time, then it's very possible that one of the researchers can end up with a lucky trial. That is exactly what happened during the 2010 World Cup.

### Paul the Octopus

Paul the Octopus was a common octopus who lived in a Sea Life Centre in Oberhausen, Germany. He is most well known for correctly guessing all seven soccer matches Germany played during the 2010 World Cup, as well as the final match, which was between Netherlands and Spain. 

Before a match was played, Paul's owners would place two boxes in his tank containing food, each box labeled with a different flag of the opposing countries. Whichever box Paul chose to eat from first was considered his prediction for the outcome of the match. 

<img src="https://news.bbcimg.co.uk/media/images/49659000/jpg/_49659323_octopus.jpg" width="400" />

So why was Paul so good at predicting the outcome of these matches? Was he actually psychic, or was he just lucky? We might ask what’s the chance he got all of the predictions correct, assuming he was just “guessing”?

Paul correctly predicted 8 of the 2010 World Cup games, each time he had a 1/2 chance of making the correct prediction. The one way to get all 8 matches correct out of 8 is:
$$(1/2)^8 = 1/256$$

So was he actually psychic? Or is there something more to uncover?

Turns out, there were tons of animals (some of them in the same zoo as Paul!) doing the same thing, trying to guess the outcome of their respective home countries' matches, including:
- Mani the Parakeet, from Singapore
- Leon the Porcupine, from Germany
- Petty the Pygmy Hippopotamus, from Germany
- Otto Armstrong the Octopus, from Germany
- Anton the Tamarin, from Germany
- Jimmy the Peruvian Guinea Pig, from Germany
- Xiaoge the Octopus, from China
- Pauline the Octopus, from the Netherlands
- Pino the Chimpanzee, from Estonia
- Apelsin the Red River Hog, from Estonia
- Harry the Crocodile, from Australia
None of whom got them all right (although Mani the Parakeet got 7 matches out of 8 right).

Some might argue that getting them all wrong would also be remarkable. So what are the chances that at least one of the 12 animals would get either all right or all wrong? 

We can use simple probability to figure this out. We have 12 trials (in this case, animals), where each independent trial has a $2*(1/2)^8 = 1/128$ chance of getting all predictions right or wrong. So what is the probability of having *at least* one success? That's $1 - P_{all \textrm{ }failures} = 1 - (127/128)^{12} = 1 - 0.910 = 0.090$

We have an 9% chance of getting an animal that will select all of the right predictions, and that's not including all of the animals in the world that were also doing these "predictions." That's not that rare - it's the dangers of multiple testing that caused this "phenomenon." This one octopus out of many different animals in the world happened to have guessed all of the right predictions, and the popularity of the situation caused it to become magical.

To those of you wondering if it really was luck, it has been shown that the species *Octopus vulgaris* is actually colorblind, and some believe that octopuses are drawn to horizontal shapes, hence Paul's decision to choose Germany, except when playing against Spain and Serbia.

In the end, we know that studies are more trustworthy when they are replicated. Data scientists should try to avoid cases like Paul the Octopus's where there has only been one real case of the animal correctly predicting a bunch of World Cup matches in a row. Only when we see him doing that for multiple soccer tournaments should we start looking at the data.

## P-Hacking is just the tip of the iceberg

As it turns out, p-hacking isn't the only thing data scientists and statisticians have to worry about when making sound inferences from data. There are many stages to the design and analysis of a successful study, as shown below (from Leek & Peng's *P values are just the tip of the iceberg*).

<img src='https://www.nature.com/polopoly_fs/7.25671.1429983882!/image/P1.jpg_gen/derivatives/landscape_300/P1.jpg'>

As shown, the last step of the whole "data pipeline" is the calculation of an inferential statistic like the p-value, and having a rule applied to it (e.g. p > 0.05). But there are many other decisions that are made beforehand, like experimental design or EDA, that can have much greater effects on the results - mistakes like simple rounding or measurement errors, choosing the wrong model, or not taking into account confounding factors can change everything. By changing the way data are cleaned, summarized, or modeled, we can achieve arbitrary levels of statistical significance.

A simple example of this would be in the case of rolling a pair of dice and getting two 6s. If we were to take a null hypothesis that the dice are fair and not weighted, and take our test statistic to be the sum of the dice, we will find that the p-value of this outcome will be 1/36 or 0.028, and gives us statistically signficant results that the dice are fair. But obviously, a single roll is not nearly enough rolls to provide us with good evidence to say whether the results are statistically significant or not, and shows that blindly applying the p-value without properly designing a good experiment can result in bad results.

In the end, what is most important is education on the subject of safe hypothesis testing, and making sure you don't fall into the follies of poor statistical decisions.